In [31]:
from rdkit import Chem
import datamol as dm 
import pandas as pd

In [32]:
master_amines = pd.read_csv("./data/master_Amines.csv")

In [33]:
smarts_str = "[#6]-[#7H2]"
smarts = dm.from_smarts(smarts_str)

In [35]:
master_amines.head(2)

,ID,Box,Type,Building Block ID,Smiles,CAS,Old Fluics,Mass (mg),Conc (M),Volume (uL),merge_timestamp
0,Amine_1,Amine_Box_1,Amine,IMA-XXXX,CC(C)N,NaN,NaN,2.5,0.5,NaN,2025-04-16T16:00:40.988684
1,Amine_2,Amine_Box_1,Amine,IMA-XXXX,CC(N)C1CCCCC1,NaN,NaN,3.4,0.5,NaN,2025-04-16T16:00:40.988684


In [52]:
def filter_on_smarts(df, smiles_col, smarts: Chem.rdchem.Mol):
    
    filtered_ndx = []
    for ndx, row in df.iterrows():
        smiles = row[smiles_col]
        
        mol = dm.to_mol(smiles)
        
        if mol.HasSubstructMatch(smarts):
            filtered_ndx.append(ndx)
            
    return df.iloc[filtered_ndx]

filtered_df = filter_on_smarts(master_amines, "Smiles", smarts)

In [53]:
filtered_df

,ID,Box,Type,Building Block ID,Smiles,CAS,Old Fluics,Mass (mg),Conc (M),Volume (uL),merge_timestamp
0,Amine_1,Amine_Box_1,Amine,IMA-XXXX,CC(C)N,NaN,NaN,2.5,0.5,NaN,2025-04-16T16:00:40.988684
1,Amine_2,Amine_Box_1,Amine,IMA-XXXX,CC(N)C1CCCCC1,NaN,NaN,3.4,0.5,NaN,2025-04-16T16:00:40.988684
2,Amine_3,Amine_Box_1,Amine,NaN,CC(N)C1CCCCC1,Placeholder,Placeholder,6.1,0.5,NaN,2025-04-16T16:00:40.988684
3,Amine_4,Amine_Box_1,Amine,IMA-XXXX,CC(N)C1=CCCC(C)C1,NaN,NaN,8.3,0.5,NaN,2025-04-16T16:00:40.988684
4,Amine_5,Amine_Box_1,Amine,IMA-XXXX,CC(N)C1=CCCC(C)C1,NaN,NaN,8.3,0.5,NaN,2025-04-16T16:00:40.988684
5,Amine_6,Amine_Box_1,Amine,NaN,NC1=C(O)C(F)=CC=C1,Placeholder,Placeholder,5.5,0.5,NaN,2025-04-16T16:00:40.988684
6,Amine_7,Amine_Box_1,Amine,NaN,NC1=NC(Cl)=C(NC=O)C(Cl)=N1,Placeholder,Placeholder,4.2,0.5,NaN,2025-04-16T16:00:40.988684
7,Amine_8,Amine_Box_1,Amine,NaN,NC1=C(C=O)C=C(Cl)C=C1,Placeholder,Placeholder,0.4,0.5,NaN,2025-04-16T16:00:40.988684
8,Amine_9,Amine_Box_1,Amine,NaN,O=CC1=C(N)N=CC=C1,Placeholder,Placeholder,4.5,0.5,NaN,2025-04-16T16:00:40.988684
9,Amine_10,Amine_Box_1,Amine,NaN,C(=O)C1=CC(Br)=CC(Br)=C1N,Placeholder,Placeholder,4.2,0.5,NaN,2025-04-16T16:00:40.988684
